# 3 - Adding and editing core information

So far, we have just highlighted functionality for the core of specifying metal and metal coordination number (metalCN).

However, if you want to only screen specific cores, or potentially screen multiple different coordination numbers this is also possible.

In this tutorial we will learn:

**(A)** How to have architector automatically fill the coordination environment with water.

**(B)** How to request specific core geometries with multiple conformers each.

**(C)** How to input a user-defined core geometry.

## For this example, we will be looking at Aqueous Nd-Chloride Salts 

Although NdCl<sub>3</sub> forms a hexahydrate, that is, NdCl<sub>3</sub>(H<sub>2</sub>O)<sub>6</sub>,
The experimental [crystal structure](https://materials.springer.com/isp/crystallographic/docs/sd_1102086) reveals a Nd atom directly coordinated by 6 H<sub>2</sub>O molecules and 2 [Cl-] molecules.

This gives enough information to come up with a possible input for Architector, starting from basic imports again!

In [ ]:
from architector import build_complex,view_structures

For visual simplicity we will define all the needed parameters in a single line for the input dictionary.

## For (A), note that simply specifying non-water ligands less that the coreCN will result in a water-filled coordination environment!

In [ ]:
inputDict = {
    'core':{'metal':'Nd','coreCN':8},
    'ligands':[
        {'smiles':'[Cl-]','coordList':[0]}, # Chloride smiles and coordList can be manually ID'ed
        {'smiles':'[Cl-]','coordList':[0]},
    ],
    'parameters':{} # Note that the default fill_ligand (filling out the coordination sphere) is H2O!
}

And we are already ready to build Aqueous Nd-Cl salt:

Note that you can ignore any numpy warnings that may arise here.

In [ ]:
out = build_complex(inputDict) # Will take a couple minutes

That was potentially a little slow! How do the structures look? 

In [ ]:
labels = list(out.keys())
view_structures(out,labels=labels)

Here, we see that a lot of these structures are quite close in structure to experiments!

So it stands to reason that running only a couple of the CN=8 structures with potentially more saved conformers might sample the configurational space with less time spent. 

## For (B), Let's pick the first 2 lowest-energy configurations to start with.

In [ ]:
labels[0:2]

The first section(s) of these labels indicates core geometries we can sample.

Here we use a list comprehension to just pull out the core geometry labels:

In [ ]:
core_types = [x.split('0')[0].strip('_') for x in labels[0:2]]
core_types

Now we can make a new input dictionary with these coreTypes as input. 

### Since we aren't sampling the full core geometry space, we can request additional conformers for the given coreTypes as an output!

In [ ]:
import copy

new_inputDict = copy.deepcopy(inputDict)

del new_inputDict['core']['coreCN'] # Remove the CN

new_inputDict['core']['coreType'] = core_types # Add in the coreTypes

new_inputDict['parameters']['n_conformers'] = 3 # Add additional 2 conformers per coreType to output!

new_inputDict

Looks good! ready for building!

In [ ]:
newout = build_complex(new_inputDict)

In [ ]:
labels = list(newout.keys())
view_structures(newout,labels=labels)

Here, we are able to sample a similar amount of the configurational space much quicker with fewer metal core symmetries sampled.

## For (C), we highlight that a user-specified core can be added.

Here, we take the exact crystal structure reported as a potential new core. (This was taken from the experimental crystal structure:

In [ ]:
from architector import convert_io_molecule,view_structures
xyzstr = """9

O        0.8878190000      1.9658050000      0.5785630000                 
O       -0.1674130000      0.3098710000      2.7398960000                 
O        3.3940290000      2.7987070000      1.3860190000                 
Nd       2.1678770000      1.0006660000      2.4350300000                 
Cl       2.3711570000     -1.0723770000      4.2983150000                 
O        3.4479350000      1.9658050000      4.2914960000                 
O        4.5031670000      0.3098710000      2.1301640000                 
O        0.9417250000      2.7987070000      3.4840410000                 
Cl       1.9645970000     -1.0723770000      0.5717450000  
"""
# mol = convert_io_molecule(xyzstr)
view_structures(xyzstr)
# Here is what this experimental core looks like! 

To translate this structure into an Architector-compatable core we will need a couple of utilites!

In [ ]:
from architector.io_molecule import convert_io_molecule # Import molecule utility included with architector
import numpy as np # Numpy for analysis

### The convert_io_molecule utility can be used to read in xyz strings, or any number of structure types and convert these into an architector molecule class, which contains ASE atoms as a subclass for utility

In [ ]:
mol = convert_io_molecule(xyzstr)
ase_atoms = mol.ase_atoms.copy()

Now, we can use the ASE atoms utilites to center the molecule on the Nd atom (Architector cores all have the metal atom at the origin (0,0,0)!):

In [ ]:
positions = ase_atoms.get_positions() # Get the xyz coordinates of the atoms
symbols = ase_atoms.get_chemical_symbols() # Get the chemical symbols
nd_index = [i for i,x in enumerate(symbols) if x == 'Nd'][0] # Pick out the Nd index
nd_posit = positions[nd_index] # Pick out the Nd position
new_positions = positions - nd_posit  # Set Nd/metal center position to (0.,0.,0.) This is the only requirement!

Now, the core structure for Architector does not include the metal, so we can remove it for the final form:

In [ ]:
new_core = [x for i,x in enumerate(new_positions) if i!=nd_index]
new_core # Now it is a new "core" centered at 0

Now we can copy and edit the inputDict with this new_core as an input:

In [ ]:
new_inputDict1 = copy.deepcopy(inputDict)

del new_inputDict1['core']['coreCN'] # Remove the CN

new_inputDict1['core']['coordList'] = new_core # Add in the user defined core!

new_inputDict1['parameters']['n_conformers'] = 5 # Add additional 4 conformers to output!

new_inputDict1['parameters']['skip_duplicate_tests'] = True # Skip duplicate checks to see all structures.

new_inputDict1

Looks good! Ready for building!

In [ ]:
newout1 = build_complex(new_inputDict1)

In [ ]:
labels = list(newout1.keys())
view_structures(newout1,labels=labels)

### Looks good! Note that architector handles the ligand assignment and symmetry determination internally as well for user-input cores!

# Conclusions

In this tutorial we learned:

**(A)** How to have architector automatically fill the coordination environment with water.

**(B)** How to request specific core geometries with multiple conformers each.

**(C)** How to input a user-defined core geometry.